# POC Power BI TS Formation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path
import glob
import xlsxwriter
import openpyxl 

## 1) Déclaration des fonctions et classes exploitées dans le code

In [2]:
class Datamanagement:
    def __init__(self):
        self.source = {} # Contient le nom et la localisation des fichiers sources
        self.dim = {} # Contient le nom et les dataframes des tables de dimension
        self.fact = {} # Contient le nom et les dataframes liés aux tables de faits
        self.changed_keys = {}
    
    def import_csv(self, filename, engine_val=None, encoding_val='utf-8', sep_val=';',low_memory_val=False):
        self.source[filename[:-4]] = pd.read_csv(r'Data/'+ filename, engine=engine_val, encoding=encoding_val, sep=sep_val, low_memory=low_memory_val)
        return self.source[filename[:-4]]
    
    def update_key(self, changed_keys, filename):
        self.changed_keys[filename] = changed_keys
        self.source[filename].rename(columns=changed_keys , inplace=True)
        return self.source[filename]
    
    def import_dim(self,dimname, dimdataframe):
        self.dim[dimname]=dimdataframe
    
    def export(self):
        for key, value in self.dim.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-8', index=False)
            
        for key, value in self.fact.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-16', index=False)
    

### Génération/Mise à jour du fichier setting

In [3]:
def import_data():
    all_dir = glob.glob("Data/*.csv")    
    all_files_name = [x[5:] for x in all_dir]
    all_files_name = [x[:-4] for x in all_files_name]
    df_files_col = pd.DataFrame()
    p = 1
    
    for dir in all_dir:
        df = pd.read_csv(dir, encoding='utf-8', sep=';')
        
        for i, col in enumerate(df.columns):
            df_files_col.at[p, 'Nom fichier source'] = dir[5:-4]
            df_files_col.at[p,'Nom champ source'] = col
            df_files_col.at[p,'Type'] = str(df[col].dtypes)
            df_files_col.at[p,'Synthèse'] = str(df[col].describe())
            df_files_col.at[p, 'Répertoire'] = dir
            p = p + i
    
    df_dir_files = df_files_col.copy()
    df_dir_files = df_dir_files[["Nom fichier source", "Répertoire"]].drop_duplicates()
    
    if path.exists("Settings & documentation\Settings.xlsx"):
        pass

    else:

        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='xlsxwriter')
        workbook  = writer.book
        df_dir_files.to_excel(writer, sheet_name='Fichiers Source', index=False)
        
        df_files_col[['Nom fichier source', 'Nom champ source', 'Type', 'Synthèse' ]].to_excel(writer, sheet_name='Fichiers et colonnes source', index=False)
                
        worksheet_FCS = writer.sheets["Fichiers et colonnes source"]
        worksheet_FS = writer.sheets["Fichiers Source"]
        #worksheet1 = workbook.add_worksheet("Mapping données")
        
        cell_format_FCS = workbook.add_format() 
        cell_format_FCS.set_text_wrap()
        cell_format_FCS.set_align('center')
        cell_format_FCS.set_align('vcenter')
        
        cell_format_FS = workbook.add_format()    
        cell_format_FS.set_align('left')
                
        worksheet_FCS.set_column('A:B', 30, cell_format_FCS)
        worksheet_FCS.set_column('C:C', 15, cell_format_FCS)
        worksheet_FCS.set_column('D:D', 25, cell_format_FCS)
        worksheet_FS.set_column('A:A', 40, cell_format_FS)
        worksheet_FS.set_column('B:B', 60, cell_format_FS)

        writer.save()
        workbook.close()

# Fonction créant la feuille Mapping dim dans le fichier Settings
# Cette feuille contient l'ensemble des valeurs de dimension et permet de réaliser un mapping pour changer le nom des attributs

def add_sheet_mapcol(DicDataframe):
    
    df = pd.DataFrame(columns=["Nom dimension", "Nom colonne"],data=[])
    
    for key, value in DicDataframe.items():        
        #if len(key) >= 3:
        #    if key[:3] == "dim":
        for i, col in enumerate(value.columns):
            df.at[i, "Nom dimension"] = key
            df.at[i, "Nom colonne"] = col
            df.at[i, "Nouveau nom"] = ""
            df.at[i, "A mapper"] = "Non"                
      
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    df.to_excel(writer, sheet_name="Mapping dim colonne",engine='openpyxl',index=False)     
    writer.save()
    writer.close()
    
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    workbook1["Mapping dim colonne"].column_dimensions["A"].width = 20
    workbook1["Mapping dim colonne"].column_dimensions["B"].width = 20
    workbook1["Mapping dim colonne"].column_dimensions["C"].width = 20
    
    writer.save()
    writer.close()
    
import_data()

my_data = Datamanagement()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (0,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
# Fonction permettant d'exporter un dataframe vers un onglet de setting avec un formatage automatique des colonnes
def export_to_settings(dataframe, sheetname):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    numbers = range(0,26,1)
    dic_letters = {} # Dictionnaire contenant les lettres de l'alphabet    
    val_max = [] # Liste contenant la taille des valeurs les plus longues
    
    # Création d'un dictionnaire stockant les lettres de l'alphabet et leur position afin d'alimenter les fonctions openpyxl
    for i in range(len(numbers)): 
        dic_letters[numbers[i]] = letters[i]
    
    # Alimentation de val_max
    for col in dataframe.columns:
        max = (dataframe[col].astype(str)).str.len().max()
        #max = dataframe[col].str.len().max()
        if max != 0:
            val_max.append(int(max)+1)
        else:
            val_max.append(15)
            
   # Ouverture du fichier setting et création de l'onglet sheetname
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    dataframe.to_excel(writer, sheet_name= sheetname, engine='openpyxl', index=False)     
    writer.save()
    writer.close()
    
   # Réouverture du fichier setting 
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    
    # Aliemntation du fichier excel avec les données du dataframe
    #for i, width in enumerate(get_col_widths(dataframe)):
    print(val_max)
    for i, width in enumerate(val_max):
        workbook1[sheetname].column_dimensions[dic_letters[i]].width = width
    
    # Sauvegarde et fermeture du fichier excel
    writer.save()
    writer.close()        

### Fonction mapping des noms de colonnes

In [5]:
# Fonction transformant le nom des colonnes suivant le mapping effectué dans l'onglet Mapping dim colonne du fichier setting
# Entrée: Dictionnaire key: Nom dataframe  Value : Dataframe
# Sortie un dictionnaire "nom"/Dataframe avec les valeurs des colonnes actualisées suivant l'onglet de mapping "Mapping dim colonne" du fichier settings

def map_col(dict_dataframe):
    md = pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name="Mapping dim colonne")
    mdf = md["Nom dimension"].unique()
    dic_map = {}
    for dim in mdf:
        map = md.loc[(md["Nom dimension"] == dim ) & (md["Nouveau nom"].notnull()== True),["Nom colonne","Nouveau nom"]]
        map.reset_index(drop=True, inplace=True)
            
        for i, val in enumerate(map.iterrows()):           
            dict_dataframe[dim].rename(columns={map.at[i, "Nom colonne"]:map.at[i,"Nouveau nom"]},inplace=True)
     
    return dict_dataframe


In [6]:
# Fonction générant une liste des valeurs des champs à mapper. Chaque type de valeur sera extraite.

def mapp_data(dict_dataframe):
    result = pd.DataFrame()
    
    try:       
        to_mapp_data = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping données") # Lecture de la feuille Mapping données
        to_mapp_data = to_mapp_data.loc[to_mapp_data["Valeurs cible"] != '']
        to_mapp_data.reset_index(drop=True, inplace=True)
        
        if to_mapp_data.empty == False:
            for i, val in enumerate(to_mapp_data.iterrows()):
                dict_dataframe[to_mapp_data.at[i,"Table"]][to_mapp_data.at[i,"Colonnes"]].replace(to_replace =to_mapp_data.at[i,"Valeurs actuelles"], 
                 value = to_mapp_data.at[i,"Valeurs cible"], inplace=True)
            
    except:
            mapp_col = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
            mapp_col = mapp_col.loc[mapp_col["A mapper"] == "Oui", ["Nom dimension", "Nom colonne", "Nouveau nom"]]
            mapp_col["Colonne"] = ""
            mapp_col.reset_index(inplace=True, drop=True)
            mapp_val = pd.DataFrame(columns=["Table", "Colonnes", "Valeurs actuelles", "Valeurs cible"])
            p=1
            if mapp_col.empty == False:
                for i, val in enumerate(mapp_col.iterrows()):
                    if str(mapp_col.at[i, "Nouveau nom"]) != "nan":
                        mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nouveau nom"]
                    else:
                         mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nom colonne"]

                for i,val in enumerate(mapp_col[["Nom dimension", "Colonne"]].iterrows()):
                    
                    if (dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).empty == False:
                        list_val = (dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                        nom_col = mapp_col.at[i, "Colonne"]
                        nom_table = mapp_col.at[i, "Nom dimension"]

                    if len(list_val) != 0:
                        for i, val in enumerate(list_val):                           
                            mapp_val.at[p,"Table"] = nom_table
                            mapp_val.at[p,"Colonnes"] = nom_col
                            mapp_val.at[p,"Valeurs actuelles"] = val
                            mapp_val.at[p,"Valeurs cible"] = ""
                            p += 1
                            
            if mapp_val.empty:                
                pass            
            else:
                export_to_settings(mapp_val,"Mapping données")            


## 2) Import des données

In [7]:
%%time
### Ajouter les tables à intégrer ici
tcw = my_data.import_csv('191119_TrainingCollectiveWishes.csv') # Demandes collectives non affectées à un plan de formation
tiw = my_data.import_csv('191119_TrainingIndividualWishes.csv') # Demandes individuelles non affectées à un plan de formation
tp = my_data.import_csv('191119_TrainingPlan.csv') # Plans de formation
tpcw = my_data.import_csv('191119_TrainingPlanCollectiveWishes.csv') # Demande de formation collectives prises en charge dans un plan de formation
tpiw = my_data.import_csv('191119_TrainingPlanIndividualWishes.csv') # Demande de formation individuelles prises en charge dans un plan de formation
tr = my_data.import_csv('191119_TrainingRegister.csv') # Table recensant les inscription aux sessions de formation 
ts = my_data.import_csv('191119_TrainingSessions.csv') # Table recensant les sessions de formation 
tsc = my_data.import_csv('191119_TrainingStageCost.csv') # Table contenant les coûts des stages
tsv2 = my_data.import_csv('191119_TrainingStagev2.csv') # Table contenant les stages
emp = my_data.import_csv('191119_Employees.csv') # Table maître employé
empc = my_data.import_csv('191119_EmployementContract.csv') # Table contrat employé
indO = my_data.import_csv('191119_IndividualOrganization.csv') # Table organisation employé
indpp =my_data.import_csv('191119_IndivPPCsNew.csv') # ?
#setting = pd.read_excel(r'Data/Settings.xlsx', sheets="Mapp_data") # Table de paramétrage servant à mapper les données

Wall time: 906 ms


## 3) Renommage des clés fonctionnelles

In [8]:
%%time
ts = my_data.update_key({'clientcode':'SessionID', 'coursecode':'FormationID', 'startdate':'SessionDate'},'191119_TrainingSessions')
tsv2 = my_data.update_key({'clientcode':'SessionID'},'191119_TrainingStagev2')
tr = my_data.update_key({'clientcode':'SessionID', 'traineusername':'USERNAME'}, '191119_TrainingRegister')
tp = my_data.update_key({'plancode':'PlanID'}, '191119_TrainingPlan')
emp = my_data.update_key({'username':'USERNAME'}, '191119_Employees')
tcw = my_data.update_key({'clientcode':'WishID','coursecode':'FormationID','username':'USERNAME'}, '191119_TrainingCollectiveWishes')
tiw = my_data.update_key({'username':'USERNAME', 'employeenumber':'ZY00.MATCLE', 'clientcode':'WishID', 'coursecode':'FormationID'},'191119_TrainingIndividualWishes')
tpcw = my_data.update_key({'plan_code':'PlanID', 'wish_code':'WishID'}, '191119_TrainingPlanCollectiveWishes')
tpiw = my_data.update_key({'plan_code':'PlanID', 'wish_code':'WishID'} ,'191119_TrainingPlanIndividualWishes')

Wall time: 39 ms


## 4) Alimentation des dictionnaires contenant les colonnes et dataframe

In [9]:
%%time

my_dict_dataframe = {"tcw": tcw, "tiw": tiw, "tp" : tp, "tpcw" : tpcw, "tr" : tr, "ts" : ts, "tsc" : tsc, "tsv2" : tsv2, "emp" : emp,
           "empc" : empc, "indO": indO, "indpp": indpp}

Wall time: 0 ns


## 5) Creation des tables de dimension

### Création de la table dimension demandes de formations

In [10]:
%%time
# Fusion des tables TrainingCollectiveWishes et TrainingIndividualWishes
dim_wish = tcw.append(tiw,sort=True)
dim_wish.reset_index(drop=True, inplace=True)
dim_wish["Wish_key"] = dim_wish.index
dim_wish.loc[dim_wish.nbtrainees.isna(), "nbtrainees"] = 1
#dim_wish = df_mapp(dim_wish)
my_data.import_dim("dim_wish_table", dim_wish)

#y_data.dim["dim_wish_table"] = dim_wish
#my_dict_dataframe["dim_wish_table"]= dim_wish
#wish_table.to_excel(r'Transformed data\wish_table.xlsx')

Wall time: 92 ms


### Traitement du mapping des colonnes et des valeurs

In [11]:
%%time
#Verification si la colonne Mapping dim existe dans le fichier setting, si non on va la créer
try:
    pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
    Mapp_sheet = True    
except:
    Mapp_sheet = False
    
if Mapp_sheet == True:    
    pass
else:
    add_sheet_mapcol(my_data.dim)

#my_dict_dataframe = map_col(my_data.dim)
#mapp_data(my_dict_dataframe)

# Mapping des données

my_data.dim = map_col(my_data.dim)
mapp_data(my_data.dim)

print(my_data.dim.keys())
      
    

dict_keys(['dim_wish_table'])
Wall time: 502 ms


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## 6) Création des tables de fait

### Table de fait demande de formation

In [12]:
%%time
#wish_fact = dim_wish[["Wish_key", "nombre de formés"]].copy()
#dim_wish.loc[:, "Nb_demandes"]= 1

#my_dict_dataframe["wish_fact"] = wish_fact

Wall time: 0 ns


## Export des données vers le répertoire Transformed data

In [13]:
%%time
#for key, value in my_dict_dataframe.items():
 #       location = "Transformed data"
  #      file_name = str(key) + '.csv'
   #     location = os.path.join(location, file_name)  
    #    value.to_csv(location, encoding='utf16', index=False)
    
my_data.export()

Wall time: 821 ms
